<a href="https://colab.research.google.com/github/FaridRash/IGNIS/blob/main/SNR_base_on_MA_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Github and Libraries



In [ ]:
!pip install "cdsapi>=0.7.4"

In [ ]:
!pip install xarray netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import cdsapi

In [ ]:
#!rm -fr /content/IGNIS

In [ ]:
!git clone https://github.com/FaridRash/IGNIS IGNIS
!rm -rf /content/IGNIS/Dataset
!rm -rf /content/IGNIS/FoV\ visualization
!rm -rf /content/IGNIS/Notebooks
!rm -rf /content/IGNIS/Reports/
!rm -rf /content/IGNIS/SNR\ Visualization

Cloning into 'IGNIS'...
remote: Enumerating objects: 293, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 293 (delta 41), reused 32 (delta 8), pack-reused 204 (from 1)
Receiving objects: 100% (293/293), 44.22 MiB | 15.41 MiB/s, done.
Resolving deltas: 100% (121/121), done.
Updating files: 100% (43/43), done.


#Dataset

In [ ]:
file_path = '/content/IGNIS/Requests/TK- PAY-MA-R.01 – Altitude and Angle/Altitude_Theta_Datetime.xlsx'
df_excel = pd.read_excel(file_path)

print(df_excel.head())


                 Datetime    Altitude      Theta
0 2029-01-02 02:43:41.770  461.218874  21.299047
1 2029-01-05 02:34:41.998  462.767619  12.416525
2 2029-01-05 02:35:11.998  463.422734  14.037020
3 2029-01-05 15:29:42.588  466.582339  22.475595
4 2029-01-05 15:30:12.588  466.012605  13.759815


In [ ]:
df_excel['Date'] = df_excel['Datetime'].dt.date
df_excel['Time'] = df_excel['Datetime'].dt.time

print(df_excel.head())

                 Datetime    Altitude      Theta        Date             Time
0 2029-01-02 02:43:41.770  461.218874  21.299047  2029-01-02  02:43:41.770000
1 2029-01-05 02:34:41.998  462.767619  12.416525  2029-01-05  02:34:41.998000
2 2029-01-05 02:35:11.998  463.422734  14.037020  2029-01-05  02:35:11.998000
3 2029-01-05 15:29:42.588  466.582339  22.475595  2029-01-05  15:29:42.588000
4 2029-01-05 15:30:12.588  466.012605  13.759815  2029-01-05  15:30:12.588000


In [ ]:
df_excel = df_excel.drop('Datetime', axis=1)

print(df_excel.head())

     Altitude      Theta        Date             Time
0  461.218874  21.299047  2029-01-02  02:43:41.770000
1  462.767619  12.416525  2029-01-05  02:34:41.998000
2  463.422734  14.037020  2029-01-05  02:35:11.998000
3  466.582339  22.475595  2029-01-05  15:29:42.588000
4  466.012605  13.759815  2029-01-05  15:30:12.588000


In [ ]:
# prompt: i want to separate time to hour minute ans second and mili second

df_excel['Hour'] = df_excel['Time'].apply(lambda x: x.hour)
df_excel['Minute'] = df_excel['Time'].apply(lambda x: x.minute)
df_excel['Second'] = df_excel['Time'].apply(lambda x: x.second)
df_excel['Microsecond'] = df_excel['Time'].apply(lambda x: x.microsecond)
df_excel = df_excel.drop(['Time', 'Minute', 'Second', 'Microsecond'], axis=1)

df_excel['Year'] = df_excel['Date'].apply(lambda x: x.year)
df_excel['Month'] = df_excel['Date'].apply(lambda x: x.month)
df_excel['Day'] = df_excel['Date'].apply(lambda x: x.day)
df_excel = df_excel.drop(['Date'], axis=1)
df_excel = df_excel.drop(['Year'], axis=1)

print(df_excel.head())


     Altitude      Theta  Hour  Month  Day
0  461.218874  21.299047     2      1    2
1  462.767619  12.416525     2      1    5
2  463.422734  14.037020     2      1    5
3  466.582339  22.475595    15      1    5
4  466.012605  13.759815    15      1    5


In [ ]:
# prompt: i need to find unics in the day

unique_days = df_excel['Day'].unique()
print("Unique days in the 'Day' column:", unique_days)

Unique days in the 'Day' column: [ 2  5  8 13 16 19 24 27  4  7 12 15 23 26  3  6 11 14 22 25 30 10 18  1
  9 17 20 28 21 29 31]


In [ ]:
df_head = df_excel.head(10).copy()
df_head

,Altitude,Theta,Hour,Month,Day
0,461.218874,21.299047,2,1,2
1,462.767619,12.416525,2,1,5
2,463.422734,14.037020,2,1,5
3,466.582339,22.475595,15,1,5
4,466.012605,13.759815,15,1,5
5,464.701066,10.172074,15,1,8
6,466.320733,17.211679,2,1,13
7,466.846238,10.164086,2,1,16
8,467.342004,18.941394,2,1,16
9,460.811780,15.738485,15,1,16


In [ ]:
df_excel['Hour'] = df_excel['Hour'].astype(str).str.zfill(2) + ':00'

print(df_excel.head())

     Altitude      Theta   Hour  Month  Day
0  461.218874  21.299047  02:00      1    2
1  462.767619  12.416525  02:00      1    5
2  463.422734  14.037020  02:00      1    5
3  466.582339  22.475595  15:00      1    5
4  466.012605  13.759815  15:00      1    5


In [51]:
df_excel['Month'] = df_excel['Month'].astype(str).str.zfill(2)

print(df_excel.head())

     Altitude      Theta   Hour Month  Day        GSD      North      South  \
0  461.218874  21.299047  02:00    01    2  76.025089  40.829036  40.828186   
1  462.767619  12.416525  02:00    01    5  76.280377  40.829036  40.828186   
2  463.422734  14.037020  02:00    01    5  76.388363  40.829036  40.828186   
3  466.582339  22.475595  15:00    01    5  76.909177  40.829036  40.828186   
4  466.012605  13.759815  15:00    01    5  76.815265  40.829036  40.828186   

        West       East  
0  14.144160  14.145534  
1  14.144157  14.145531  
2  14.144156  14.145530  
3  14.144150  14.145524  
4  14.144151  14.145525  


##Test

In [ ]:
df10 = df_excel.head(10).copy()

In [ ]:
def compute_gsd(altitude_m):
  pixel_pitch_m = pixel_pitch = 12e-6        #12 micrometers
  focal_length_m = 0.0728                    # 72.8 mm
  return (altitude_m * pixel_pitch_m) / focal_length_m


In [ ]:
def shift_longitude_west(base_lon, lat_deg, distance_m):
    """
    Shift a longitude west by a distance in meters at a given latitude.

    Parameters:
    - base_lon: original longitude in degrees
    - lat_deg: latitude in degrees
    - distance_m: westward distance in meters

    Returns:
    - new longitude shifted westward (float)
    """
    meters_per_deg_lon = 111320 * np.cos(np.radians(lat_deg))
    delta_deg = distance_m / meters_per_deg_lon
    return base_lon - delta_deg


In [ ]:
df_excel['GSD'] = df_excel.apply(lambda row: compute_gsd(row['Altitude']*1000), axis=1)

print(df_excel.head())

     Altitude      Theta   Hour  Month  Day        GSD
0  461.218874  21.299047  02:00      1    2  76.025089
1  462.767619  12.416525  02:00      1    5  76.280377
2  463.422734  14.037020  02:00      1    5  76.388363
3  466.582339  22.475595  15:00      1    5  76.909177
4  466.012605  13.759815  15:00      1    5  76.815265


In [ ]:
"""
North: 40.829036
South: 40.828186
West: 14.145063
East: 14.146437
"""

'\nNorth: 40.829036\nSouth: 40.828186\nWest: 14.145063\nEast: 14.146437\n'

In [ ]:
df_excel['North'] = 40.829036
df_excel['South'] = 40.828186
df_excel['West'] = 14.145063
df_excel['East'] = 14.146437

print(df_excel.head())

     Altitude      Theta   Hour  Month  Day        GSD      North      South  \
0  461.218874  21.299047  02:00      1    2  76.025089  40.829036  40.828186   
1  462.767619  12.416525  02:00      1    5  76.280377  40.829036  40.828186   
2  463.422734  14.037020  02:00      1    5  76.388363  40.829036  40.828186   
3  466.582339  22.475595  15:00      1    5  76.909177  40.829036  40.828186   
4  466.012605  13.759815  15:00      1    5  76.815265  40.829036  40.828186   

        West       East  
0  14.145063  14.146437  
1  14.145063  14.146437  
2  14.145063  14.146437  
3  14.145063  14.146437  
4  14.145063  14.146437  


In [ ]:
def shift_west_by_gsd(row):
    """
    Shifts the West and East longitude values westward by the amount corresponding
    to the GSD (Ground Sample Distance) in meters.

    Parameters:
        row (pd.Series): A row from the DataFrame with 'GSD', 'North', 'South', 'West', and 'East' fields.

    Returns:
        pd.Series: New values for 'West_shifted' and 'East_shifted'.
    """
    # Calculate center latitude in radians
    center_lat = (row['North'] + row['South']) / 2
    lat_rad = np.radians(center_lat)

    # Convert GSD in meters to degrees longitude
    delta_lon_deg = row['GSD'] / (111320 * np.cos(lat_rad))

    # Apply the westward shift
    row['West'] = row['West'] - delta_lon_deg
    row['East'] = row['East'] - delta_lon_deg

    return row['West'], row['East']

In [ ]:
df_excel[['West', 'East']] = df_excel.apply(lambda row: pd.Series(shift_west_by_gsd(row)), axis=1)


In [ ]:
print(df_excel.head())

     Altitude      Theta   Hour  Month  Day        GSD      North      South  \
0  461.218874  21.299047  02:00      1    2  76.025089  40.829036  40.828186   
1  462.767619  12.416525  02:00      1    5  76.280377  40.829036  40.828186   
2  463.422734  14.037020  02:00      1    5  76.388363  40.829036  40.828186   
3  466.582339  22.475595  15:00      1    5  76.909177  40.829036  40.828186   
4  466.012605  13.759815  15:00      1    5  76.815265  40.829036  40.828186   

        West       East  
0  14.144160  14.145534  
1  14.144157  14.145531  
2  14.144156  14.145530  
3  14.144150  14.145524  
4  14.144151  14.145525  


#Fetch the temperature

In [ ]:
# prompt: install cdsapi

!pip install "cdsapi>=0.7.4"

In [ ]:
cdsapirc = """
url: https://cds.climate.copernicus.eu/api
key: d97ec102-82f7-4827-8b82-13721a50119b

"""

with open('/root/.cdsapirc', 'w') as f:
    f.write(cdsapirc)

In [50]:
dataset = "reanalysis-era5-single-levels"
for index, row in df_excel.iterrows():
    month_str = str(row['Month']).zfill(2)
    request = {
      "product_type": ["reanalysis"],
      "variable": ["skin_temperature"],
      "year": ["2024"],
      "month": [
          "01", "02", "03",
          "04", "05", "06",
          "07", "08", "09",
          "10", "11", "12"
      ],
      "day": [
          "01", "02", "03",
          "04", "05", "06",
          "07", "08", "09",
          "10", "11", "12",
          "13", "14", "15",
          "16", "17", "18",
          "19", "20", "21",
          "22", "23", "24",
          "25", "26", "27",
          "28", "29", "30",
          "31"
      ],
      "time": [
          "02:00", "03:00", "14:00",
          "15:00", "16:00"
      ],
      "area": [90, -180, -90, 180],
      "data_format": "netcdf",
      "download_format": "unarchived"
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

📦 Requesting: Row 0 → /content/temp_0.nc
⚠️ Error on row 0: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-single-levels

📦 Requesting: Row 1 → /content/temp_1.nc
⚠️ Error on row 1: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-single-levels

📦 Requesting: Row 2 → /content/temp_2.nc
⚠️ Error on row 2: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-single-levels

📦 Requesting: Row 3 → /content/temp_3.nc
⚠️ Error on row 3: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-single-levels

📦 Requesting: Row 4 → /content/temp_4.nc
⚠️ Error on row 4: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-single-levels

📦 Requesting: Row 5 → /content/temp_5.nc
⚠️ E